In [1]:
%load_ext autoreload
%autoreload 2

%config IPython.sys.argv=['-Xfrozen_modules=off']

from qiskit_metal import draw, Dict, designs, MetalGUI
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
import qiskit_metal as metal
# ?metal.qlibrary.core.QComponent.add_qgeometry

design = metal.designs.design_planar.DesignPlanar()
gui = metal.MetalGUI(design)
design.overwrite_enabled = True

# dir(QComponent)
cpw_width = '11.7um'

design._chips.main.size.size_x = '5mm'
design._chips.main.size.size_y = '5mm'
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

from collections import OrderedDict

import numpy as np

In [2]:
xmon_options = dict(
    connection_pads=dict(
        c = dict(
                #  claw_length = '215um',
                #  ground_spacing = '10um', 
                 claw_gap = '5.1um', 
                 claw_width = '15um',),
    ),
    cross_width = '30um',
    # cross_length = '240um',
    cross_gap = '30um',
    aedt_hfss_inductance = 12E-9
)

# Create a new Transmon Cross object with name 'Q1'
q = TransmonCross(design, 'Q', options=xmon_options)

gui.rebuild()  # line needed to propagate the updates from the qubit instance into the junction design table
gui.autoscale()

# LOM sim

In [3]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [ ]:
c1.sim.setup.reuse_selected_design = False
c1.sim.setup.reuse_setup = False

# example: update single setting
c1.sim.setup.max_passes = 30
c1.sim.setup.min_converged_passes = 1
c1.sim.setup.percent_error = 0.1
c1.sim.setup.name = 'sweep_setup'


In [4]:
def render_design():
    c1.sim.render_design()

In [5]:
def run_simulation():
    c1.sim.setup.reuse_selected_design = False
    c1.sim.setup.reuse_setup = False

    # example: update single setting
    c1.sim.setup.max_passes = 30
    c1.sim.setup.min_converged_passes = 1
    c1.sim.setup.percent_error = 0.1
    c1.sim.setup.name = 'sweep_setup'

    c1.sim.add_setup(name = c1.sim.setup.name)
    
    Lj = design.components['Q'].options['aedt_hfss_inductance']
    design.components['Q'].options['hfss_inductance'] = Lj
    design.rebuild()
    
    c1.sim.analyze_setup(name = c1.sim.setup.name)

def add_project_level_vars(renderer):
    Lj = design.components['Q'].options['aedt_hfss_inductance']
    design.components['Q'].options['hfss_inductance'] = Lj
    design.rebuild()

def run_analysis():
    cap_df = c1.sim.capacitance_matrix
    cap_df['misc'] = [ground_spacing_arr[i], cross_len_arr[j], claw_len_arr[k]]
    return cap_df

In [ ]:
import pandas as pd

ground_spacing_arr = [3, 4, 5, 7, 9, 15, 22, 30]
cross_len_arr = [50+i*50 for i in range(10)]
claw_len_arr = [10+i*50 for i in range(10)]

all_cap_matricies = []

# c1.sim.add_seutp(name = <setup_name>, ...)


for i in range(len(ground_spacing_arr)):
    for j in range(len(cross_len_arr)):
        for k in range(j+1):
            design.components['Q'].options['connection_pads']['c']['ground_spacing'] = str(ground_spacing_arr[i]) + 'um'
            design.components['Q'].options['cross_length'] = str(cross_len_arr[j]) + 'um'
            design.components['Q'].options['connection_pads']['c']['claw_length'] = str(claw_len_arr[k]) + 'um'
            
            Lj = design.components['Q'].options['aedt_hfss_inductance']
            design.components['Q'].options['hfss_inductance'] = Lj
            design.rebuild()

            # c1.sim.render_design(...)
            # c1.sim.analyze_setup(name = <setup_name>...)

            c1.sim.run(name = 'sweep_v1.0', components=['Q'],
               open_terminations=[('Q', 'c')])
            cap_df = c1.sim.capacitance_matrix
            cap_df['misc'] = [ground_spacing_arr[i], cross_len_arr[j], claw_len_arr[k]]
            all_cap_matricies.append(cap_df)

            print(f'For Qubit_{ground_spacing_arr[i]}_{cross_len_arr[j]}_{claw_len_arr[k]}, the cap matrix is...')
            print(cap_df)

final_df = pd.concat(all_cap_matricies)

In [ ]:
project_name = c1.sim.renderer.pinfo.project_name
design_name = c1.sim.renderer.get_active_design_name()

In [ ]:
from pyaedt import Q3d

app = Q3d()

# app.materials['silicon'].permittivity = 11.45

In [6]:
def run_single_design(): # this will be used as `custom_analysis`

    render_design()
    run_simulation()
    package = run_analysis()
    return package

In [7]:
from metal_library.sweeper import QSweeper
sweeper = QSweeper(design)

In [8]:
sweep_parameters = dict(
  connection_pads = dict(
    readout = dict(
      connector_type = '0', 
      claw_length = [f'{10+i*50}um' for i in range(10)],
      ground_spacing = ['3um', '4um', '5um', '7um', '9um', '15um', '22um', '30um'], 
      claw_gap = ['5.1um'], 
      claw_width = ['15um']
      )
    ),
  cross_width = ['30um'],
  cross_length = [f'{50+i*50}um' for i in range(10)],
  cross_gap = ['30um'],
  aedt_hfss_inductance = [12E-9]
)
my_librarian = sweeper.run_single_component_sweep(component_name='Q',
                                                  parameters = sweep_parameters,
                                                  custom_analysis = run_single_design,
                                                  parameters_slice=slice(1,2),
                                                  save_path="./sweep_08_19_test.csv")

AttributeError: 'QSweeper' object has no attribute 'run_single_component_sweep'

In [9]:
dir(sweeper)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'design',
 'run_sweep',
 'update_qcomponent']